In [17]:
import pandas as pd
import nltk 
import sys

In [18]:
PATH = 'pbp_2019.xlsx'
data = pd.read_excel(PATH,sheet_name='predict_touchdown')

In [19]:
# clean data if it NaN
data = data.dropna()
# make dictionary
teams = data['OffenseTeam'].unique().tolist()
teams.sort()
nums = list(range(0,32))
# print(teams)
# print(nums)

team_label = dict(zip(teams, nums))
print(team_label)

plays = data['PlayType'].unique().tolist()
plays.sort()
nums = list(range(0,len(plays)))

play_label = dict(zip(plays, nums))
print(play_label)

{'ARI': 0, 'ATL': 1, 'BAL': 2, 'BUF': 3, 'CAR': 4, 'CHI': 5, 'CIN': 6, 'CLE': 7, 'DAL': 8, 'DEN': 9, 'DET': 10, 'GB': 11, 'HOU': 12, 'IND': 13, 'JAX': 14, 'KC': 15, 'LA': 16, 'LAC': 17, 'MIA': 18, 'MIN': 19, 'NE': 20, 'NO': 21, 'NYG': 22, 'NYJ': 23, 'OAK': 24, 'PHI': 25, 'PIT': 26, 'SEA': 27, 'SF': 28, 'TB': 29, 'TEN': 30, 'WAS': 31}
{'CLOCK STOP': 0, 'EXCEPTION': 1, 'EXTRA POINT': 2, 'FIELD GOAL': 3, 'FUMBLES': 4, 'KICK OFF': 5, 'NO PLAY': 6, 'PASS': 7, 'PENALTY': 8, 'PUNT': 9, 'QB KNEEL': 10, 'RUSH': 11, 'SACK': 12, 'SCRAMBLE': 13, 'TWO-POINT CONVERSION': 14}


In [20]:
# replace teams and playtype with ints
data1 = data.applymap(lambda x: team_label.get(x) if x in team_label else x)
data2 = data1.applymap(lambda x: play_label.get(x) if x in play_label else x)
df = data2

In [114]:
print(df.iloc[60:100])

     OffenseTeam  DefenseTeam  PlayType  IsTouchdown
61            30           21         7            1
62            30           21         7            0
63            30           21        11            0
64            30           21        11            0
65            30           21        11            0
66            30           21        11            0
67            30           21         9            0
68            21           30         2            0
69            21           30         7            1
70            21           30         7            0
71            21           30        11            0
72            21           30        11            0
73            30           21         2            0
74            30           21         7            1
75            30           21        11            0
76            30           21         7            0
77            11           19        10            0
78            11           19        10       

In [75]:
# https://stackoverflow.com/questions/44747343/keras-input-explanation-input-shape-units-batch-size-dim-etc
# https://www.tensorflow.org/guide/keras/train_and_evaluate

# use part of the dataset to create inputs and outputs
import numpy as np

df2 = df.iloc[:30000]
x_train = df2[['OffenseTeam','DefenseTeam','PlayType']].to_numpy()

df3 = df.iloc[30000:]
x_test = df3[['OffenseTeam','DefenseTeam','PlayType']].to_numpy()
y_test = df3['IsTouchdown']

# reshape inputs
x_train = x_train.reshape(30000,3)
x_test = x_test.reshape(9047,3)

# reserve 5000 for validation
x_val = x_train[-5000:]
y_val = y_train[-5000:]
x_train = x_train[:-5000]
y_train = y_train[:-5000]


In [76]:
# create keras model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from keras.layers import concatenate

# define input
tensor = Input(shape=(3 ,)) # input shape is 3 integers (OT, DT, PT)

# create the layers and pass them the input tensor to get the output tensor:    
hidden1Out = Dense(units=4)(tensor)    
hidden2Out = Dense(units=4)(hidden1Out)    
finalOut = Dense(units=1)(hidden2Out) 

# define the model's start and end points    
model = Model(tensor,finalOut)

In [85]:
# compile
model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss='binary_crossentropy',
    # List of metrics to monitor
    metrics=['accuracy'],
)


In [86]:
# fit the trianing data
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val)

)

Epoch 1/2
391/391 [==============================] - 0s 941us/step - loss: 0.7324 - accuracy: 0.9525 - val_loss: 0.1542 - val_accuracy: 0.9900
Epoch 2/2
391/391 [==============================] - 0s 719us/step - loss: 0.7324 - accuracy: 0.9525 - val_loss: 0.1542 - val_accuracy: 0.9900


In [138]:
# evaluate moel on test data
results = model.evaluate(x_test, y_test, batch_size=128)

# save keras model
model.save('model')

71/71 [==============================] - 0s 608us/step - loss: 0.1534 - accuracy: 0.9901
INFO:tensorflow:Assets written to: model\assets


In [141]:
# note that the output of a neural network will never be binary

# also note that the same order of OT,DT,PT can produce different outputs at different points in the game, so it is difficult to  maintain accuracy

# estimate a threshold for the prediction value
def get_prediction(model,OT,DT,PT):
    test_array = [OT,DT,PT] 
    test_array = np.asarray(test_array)
    input_array = test_array[None,:]
    prediction = model.predict(input_array)
    print(prediction)
    if prediction < -20:
        print("touchdown!!!")
    else:
        print("no score :(")
    return

def main():
    # testing predictions
    import os
    PATH = os.getcwd()
    PATH=PATH+r'\model'
    model = keras.models.load_model(PATH)
    get_prediction(model,5,15,9)
    
if __name__ == '__main__':
    main()

[[-8.256]]
no score :(
